<a href="https://colab.research.google.com/github/keertu24/databricks_1/blob/main/72_Explain_Plan_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ColabSpark") \
    .getOrCreate()


In [3]:
employees_data = [
    (1, "James",   "Sales",      3000),
    (2, "Michael", "Sales",      4600),
    (3, "Robert",  "Sales",      4100),
    (4, "Maria",   "Finance",    3000),
    (5, "Scott",   "Finance",    3300),
    (6, "Jen",     "Finance",    3900),
    (7, "Jeff",    "Marketing",  3000),
    (8, "Kumar",   "Marketing",  2000),
    (9, "Sara",    "Marketing",  3800)
]

employees_cols = ["emp_id", "name", "department", "salary"]
df_emp = spark.createDataFrame(employees_data, employees_cols)
df_emp.show()


+------+-------+----------+------+
|emp_id|   name|department|salary|
+------+-------+----------+------+
|     1|  James|     Sales|  3000|
|     2|Michael|     Sales|  4600|
|     3| Robert|     Sales|  4100|
|     4|  Maria|   Finance|  3000|
|     5|  Scott|   Finance|  3300|
|     6|    Jen|   Finance|  3900|
|     7|   Jeff| Marketing|  3000|
|     8|  Kumar| Marketing|  2000|
|     9|   Sara| Marketing|  3800|
+------+-------+----------+------+



In [4]:
department_data = [
    ("Sales",     "Region_A"),
    ("Finance",   "Region_B"),
    ("Marketing", "Region_C"),
    ("HR",        "Region_D")   # unused, forces LEFT JOIN logic
]

department_cols = ["department", "region"]
df_dept = spark.createDataFrame(department_data, department_cols)
df_dept.show()


+----------+--------+
|department|  region|
+----------+--------+
|     Sales|Region_A|
|   Finance|Region_B|
| Marketing|Region_C|
|        HR|Region_D|
+----------+--------+



In [5]:
df_join = df_emp.join(df_dept, "department", "left")
df_join.show()


+----------+------+-------+------+--------+
|department|emp_id|   name|salary|  region|
+----------+------+-------+------+--------+
|     Sales|     1|  James|  3000|Region_A|
|     Sales|     2|Michael|  4600|Region_A|
|     Sales|     3| Robert|  4100|Region_A|
|   Finance|     4|  Maria|  3000|Region_B|
|   Finance|     5|  Scott|  3300|Region_B|
|   Finance|     6|    Jen|  3900|Region_B|
| Marketing|     7|   Jeff|  3000|Region_C|
| Marketing|     8|  Kumar|  2000|Region_C|
| Marketing|     9|   Sara|  3800|Region_C|
+----------+------+-------+------+--------+



In [6]:
from pyspark.sql.functions import sum as sum_, avg, count, max as max_

df_agg = (
    df_join.groupBy("region")
           .agg(
               count("*").alias("employee_count"),
               sum_("salary").alias("total_salary"),
               avg("salary").alias("avg_salary"),
               max_("salary").alias("max_salary")
           )
)

df_agg.show()


+--------+--------------+------------+------------------+----------+
|  region|employee_count|total_salary|        avg_salary|max_salary|
+--------+--------------+------------+------------------+----------+
|Region_C|             3|        8800|2933.3333333333335|      3800|
|Region_A|             3|       11700|            3900.0|      4600|
|Region_B|             3|       10200|            3400.0|      3900|
+--------+--------------+------------+------------------+----------+



In [7]:
df_final = df_agg.orderBy("total_salary", ascending=False)
df_final.show()


+--------+--------------+------------+------------------+----------+
|  region|employee_count|total_salary|        avg_salary|max_salary|
+--------+--------------+------------+------------------+----------+
|Region_A|             3|       11700|            3900.0|      4600|
|Region_B|             3|       10200|            3400.0|      3900|
|Region_C|             3|        8800|2933.3333333333335|      3800|
+--------+--------------+------------+------------------+----------+



In [8]:
df_final.explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [total_salary#43L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(total_salary#43L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=670]
      +- HashAggregate(keys=[region#18], functions=[count(1), sum(salary#3L), avg(salary#3L), max(salary#3L)])
         +- Exchange hashpartitioning(region#18, 200), ENSURE_REQUIREMENTS, [plan_id=667]
            +- HashAggregate(keys=[region#18], functions=[partial_count(1), partial_sum(salary#3L), partial_avg(salary#3L), partial_max(salary#3L)])
               +- Project [salary#3L, region#18]
                  +- SortMergeJoin [department#2], [department#17], LeftOuter
                     :- Sort [department#2 ASC NULLS FIRST], false, 0
                     :  +- Exchange hashpartitioning(department#2, 200), ENSURE_REQUIREMENTS, [plan_id=659]
                     :     +- Project [department#2, salary#3L]
                     :        +- Scan ExistingRDD[emp_id#0L,name#